In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from aging.plotting import format_plots, PlotConfig, save_factory, figure, legend, format_pizza_plots
from collections import Counter
from matplotlib.lines import Line2D
from tqdm import tqdm

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from matplotlib.gridspec import GridSpec
from collections import defaultdict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import LinearSVC
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
import random

In [3]:
def mm_norm_col(column):
    return (column - column.min()) / (column.max() - column.min())

In [4]:
keep_syllables = np.loadtxt('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/to_keep_syllables_raw.txt', dtype=int)

df = pd.read_parquet('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/longtogeny_v2_males_raw_usage_matrix_v00.parquet').astype(float)
df = df[keep_syllables].groupby(['age','uuid','mouse']).mean()

ages= df.index.get_level_values('age')
weeks = (ages * 7) // 7
df['binned_age'] = weeks
sample = df.groupby(['mouse','binned_age']).first().reset_index()
#sample2 = df.groupby(['mouse','binned_age']).last().reset_index()
#sample2['binned_age'] = sample2['binned_age']+0.5
#df_male = pd.concat([sample,sample2])
df_male = sample.copy()
df_male.rename(columns={'binned_age': 'age'}, inplace=True)

df_male.set_index(['age', 'mouse'], inplace=True)

In [5]:
df_male.groupby('age').size()

age
3.0     20
4.0     20
5.0     20
6.0     20
7.0     20
8.0     20
9.0     20
10.0    20
11.0    20
12.0    20
13.0    20
14.0    20
15.0    20
16.0    20
17.0    20
18.0    20
19.0    20
20.0    20
21.0    20
22.0    20
23.0    19
24.0    19
25.0    17
26.0    20
27.0    20
29.0    19
30.0    20
31.0    20
32.0    20
33.0    20
34.0    19
36.0    19
37.0    20
38.0    19
39.0    20
40.0    18
41.0    19
42.0    20
43.0    20
44.0    20
45.0    17
46.0    18
47.0    19
48.0    19
49.0    18
50.0    19
dtype: int64

In [6]:
# avarage per age or session
m_df= df_male.groupby(['age','mouse']).mean()
avg_m_df = m_df.groupby(['age']).mean()

# normalize the data
m_norm = mm_norm_col(m_df)
avg_m_norm = mm_norm_col(avg_m_df)

In [7]:
df_indv=m_norm.groupby(['mouse','age']).mean()
df=df_indv.copy().reset_index()

In [8]:
# choose a model
clf = svm.SVC(kernel='linear') 

# choose cross validation scheme
cv = LeaveOneOut()

#number of iterations for shuffle
it=100

In [9]:
# prepare colors for plot
from matplotlib.colors import LinearSegmentedColormap
colors = ['#c7eae5','#008C8D']
import matplotlib
cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
matplotlib.cm.register_cmap("dana", cmap)
pl = sns.color_palette("dana", n_colors=50)

/tmp/ipykernel_3711/3294620671.py:6: MatplotlibDeprecationWarning: The register_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps.register(name)`` instead.
  matplotlib.cm.register_cmap("dana", cmap)


In [10]:
xmale = pd.DataFrame()
a = 0
n=8
nmin=8
age = []
acc = []
sh_acc = []
sh_ages = []
coefficients_list = []

while True:
    # Build the current age matrix
    for m, _df in enumerate(df.groupby('mouse')):
        xmale_temp = _df[1][a:a+n]
        xmale = pd.concat([xmale, xmale_temp])
        
    rep_counts = xmale.groupby('mouse')['age'].size()
    keep_mice = list(rep_counts[rep_counts >= nmin].index)  # keep mice that have at least nmin sessions
    xmale = xmale[xmale['mouse'].isin(keep_mice)]
    if xmale.empty:
        break
    age.append(np.floor(np.mean(xmale['age'])).astype(int))    
    ymale = xmale['mouse']
    x = xmale.drop(['mouse', 'age'], axis=1).to_numpy()
    #x=np.log(x+1e-6)
    # Fit the model and collect coefficients
    clf.fit(x, ymale)
    coefficients_list.append(np.mean(clf.coef_,axis=0))
    
    # Perform cross-validation
    y_pred = cross_val_predict(clf, x, ymale, cv=cv)
    temp_acc = np.mean(cross_val_score(clf, x, ymale, cv=cv, n_jobs=-1))
    acc.append(temp_acc)
    
    # Shuffle labels and calculate shuffled accuracy
    for i in tqdm(range(it)):
        ysh_temp = np.random.permutation(ymale)
        y_pred_temp = cross_val_score(clf, x, ysh_temp, cv=cv, n_jobs=-1)
        sh_acc.append(np.mean(y_pred_temp))
        sh_ages.append(np.floor(np.mean(xmale['age'])).astype(int))
    
    # Handle variables for the loop
    xmale = pd.DataFrame()
    a = a + 1


 20%|█████████████████████████████████▌                                                                                                                                      | 20/100 [00:17<01:10,  1.13it/s]

In [11]:
# create df for plotting
df_sh = pd.DataFrame()
df_sh['acc']=sh_acc
df_sh['ages'] = sh_ages

df = pd.DataFrame()
df['acc']=acc
df['ages'] = age

In [12]:
format_plots()

In [13]:
fig, ax = plt.subplots(figsize=(1.3, 1.3))
sns.pointplot(data=df_sh, x="ages", y="acc", ax=ax, color='grey',estimator='mean', errorbar=('sd',2),join=True)
plt.setp(ax.collections, alpha=.3) #for the markers
plt.setp(ax.lines, alpha=.3)       #for the lines
sns.pointplot(data=df, x="ages", y="acc",
              dodge=0, 
              join=True, 
              ax=ax, 
              scale=0.5,
              hue='ages',
              palette=pl,
              estimator='mean', errorbar=('sd',2),
              #edgecolor='gray',
              #linewidth=0.3
             )
plt.legend([],[], frameon=False)
#plt.xticks([])
ax.set_ylim([0,1.1])
# Customize xticks
xticks = [6, 16, 26, 36]
ax.set_xticks(xticks)

#ax.set_xticks([0,11,31,49,67])
#ax.set_xticklabels([10,20,30,40])

sns.despine()
#ax.set_title('identity classifier male')
c = PlotConfig()

In [14]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'indv_decoder_longv2_male_norm.pdf')

In [15]:
fig, ax = plt.subplots(figsize=(1.3, 1.3))
# Convert coefficients list to a DataFrame
coefficients_df = pd.DataFrame(coefficients_list)
sns.heatmap(coefficients_df.T, cmap='coolwarm', center=0, annot=False, fmt='.2f', vmin=-0.2, vmax=0.2)
plt.ylabel('Syllables')
plt.xlabel('Ages')

# Set x-axis ticks to show only every 6 ages
tick_positions = range(0, len(age), 6)
tick_labels = [age[i] for i in tick_positions]
plt.xticks(tick_positions, tick_labels)

plt.show()

In [16]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'indv_decoder_longv2_coefficient_male_norm.pdf')

In [17]:
from sklearn.feature_selection import mutual_info_classif as MIC
from sklearn.feature_selection import mutual_info_regression as MIR
X = df_indv.to_numpy()
y = list(df_indv.index.get_level_values('mouse'))
mi_score_id = MIC(X,y)
id_indx = np.argsort(mi_score_id)[::-1][0:len(keep_syllables)] # syllble index
impsyl_id= list(df_indv.columns[id_indx]) #syllable id

In [18]:
# Convert coefficients list to a DataFrame
coef= pd.DataFrame(coefficients_list).T
coef['syll'] = keep_syllables
coef.set_index(['syll'],inplace=True)
# Step 3: Rename the columns
coef.columns = age
coef = coef.reindex(impsyl_id)

In [19]:
fig, ax = plt.subplots(figsize=(1.3, 1.3))
sns.heatmap(coef, cmap='coolwarm', center=0, annot=False, fmt='.2f', vmin=-0.2, vmax=0.2)
plt.ylabel('Syllables')
plt.xlabel('Ages')

# Set x-axis ticks to show only every 6 ages
tick_positions = range(0, len(age), 6)
tick_labels = [age[i] for i in tick_positions]
plt.xticks(tick_positions, tick_labels)
plt.show()

In [20]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'indv_decoder_longv2_coefficient_resorted_male_norm.pdf')

In [21]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Define the number of colors in the tab20 colormap
num_colors = 20

# Create a figure and an axis
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

# Create a colormap object
cmap = plt.get_cmap('tab20', num_colors)

# Create a ScalarMappable with the colormap and normalize it
norm = mcolors.Normalize(vmin=0, vmax=num_colors-1)
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

# Add a color bar to the plot
cbar = fig.colorbar(sm, orientation='horizontal', ticks=range(num_colors))

# Set the color bar ticks and labels
cbar.ax.set_xticklabels([str(i) for i in range(1, num_colors+1)])
cbar.set_label('tab20 Colormap')

# Show the plot
plt.show()

In [22]:
c = PlotConfig()
#fig.savefig(c.dana_save_path / "fig3/ 'longv2_indv_colormap.pdf')